In [25]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
Candledf=pd.read_csv('../result.dat',low_memory=False)
Candledf['ndatetime']=pd.to_datetime(Candledf['ndatetime'],format='%Y-%m-%d %H:%M:%S.%f')
Candledf.tail()

,ndatetime,open,high,low,close,volume
10037,2021-10-29 11:24:05,16943,16958,16936,16957,12000
10038,2021-10-29 12:00:00,16956,16976,16943,16960,12000
10039,2021-10-29 12:30:05,16960,16965,16938,16944,12000
10040,2021-10-29 13:11:58,16944,16965,16940,16945,12000
10041,2021-10-29 13:35:42,16946,16958,16943,16945,6758


In [26]:
count = 87
signals = pd.DataFrame(index=Candledf.index)
signals['trend'] = Candledf['close']
signals['signal'] = 0
signals['RollingMax'] = (signals.trend.shift(1).rolling(count).max())
signals['RollingMin'] = (signals.trend.shift(1).rolling(count).min())
signals.loc[signals['RollingMax'] < signals.trend,'signal'] = 1
signals.loc[signals['RollingMin'] > signals.trend,'signal'] = -1


In [27]:
def buy_stock(
    real_movement,
    signal,
    initial_money = 100000,
    max_buy = 1,
    max_sell = 1,
):
    """
    real_movement = actual movement in the real world
    delay = how much interval you want to delay to change our decision from buy to sell, vice versa
    initial_state = 1 is buy, 0 is sell
    initial_money = 1000, ignore what kind of currency
    max_buy = max quantity for share to buy
    max_sell = max quantity for share to sell
    """
    starting_money = initial_money
    states_sell = []
    states_buy = []
    current_inventory = 0

    def buy(i, initial_money, current_inventory):
        shares = initial_money // real_movement[i]
        if shares < 1:
            print(
                'day %d: total balances %f, not enough money to buy a unit price %f'
                % (i, initial_money, real_movement[i])
            )
        else:
            if shares > max_buy:
                buy_units = max_buy
            else:
                buy_units = shares
            initial_money -= buy_units * real_movement[i]
            current_inventory += buy_units
            print(
                'day %d: buy %d units at price %f, total balance %f'
                % (i, buy_units, buy_units * real_movement[i], initial_money)
            )
            states_buy.append(0)
        return initial_money, current_inventory

    for i in range(real_movement.shape[0] - int(0.025 * len(Candledf))):
        state = signal[i]
        if state == 1:
            initial_money, current_inventory = buy(
                i, initial_money, current_inventory
            )
            states_buy.append(i)
        elif state == -1:
            if current_inventory == 0:
                    print('day %d: cannot sell anything, inventory 0' % (i))
            else:
                if current_inventory > max_sell:
                    sell_units = max_sell
                else:
                    sell_units = current_inventory
                current_inventory -= sell_units
                total_sell = sell_units * real_movement[i]
                initial_money += total_sell
                try:
                    invest = (
                        (real_movement[i] - real_movement[states_buy[-1]])
                        / real_movement[states_buy[-1]]
                    ) * 100
                except:
                    invest = 0
                print(
                    'day %d, sell %d units at price %f, investment %f %%, total balance %f,'
                    % (i, sell_units, total_sell, invest, initial_money)
                )
            states_sell.append(i)
            
    invest = ((initial_money - starting_money) / starting_money) * 100
    total_gains = initial_money - starting_money
    return states_buy, states_sell, total_gains, invest

In [28]:

states_buy, states_sell, total_gains, invest = buy_stock(Candledf.close, signals['signal'])

day 98: buy 1 units at price 10693.000000, total balance 89307.000000
day 100: buy 1 units at price 10694.000000, total balance 78613.000000
day 102: buy 1 units at price 10696.000000, total balance 67917.000000
day 121: buy 1 units at price 10705.000000, total balance 57212.000000
day 122: buy 1 units at price 10727.000000, total balance 46485.000000
day 138: buy 1 units at price 10738.000000, total balance 35747.000000
day 140: buy 1 units at price 10765.000000, total balance 24982.000000
day 142: buy 1 units at price 10780.000000, total balance 14202.000000
day 145: buy 1 units at price 10782.000000, total balance 3420.000000
day 149: total balances 3420.000000, not enough money to buy a unit price 10783.000000
day 150: total balances 3420.000000, not enough money to buy a unit price 10795.000000
day 153: total balances 3420.000000, not enough money to buy a unit price 10802.000000
day 155: total balances 3420.000000, not enough money to buy a unit price 10806.000000
day 156: total 